In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import pickle
from sklearn.metrics import mean_squared_error

## RMSE computed only with ingredients that are in recipe

In [2]:
test_predictions = pickle.load(open("test_predictions_hidden1.pkl", "rb"))
hidden_rankings = pickle.load(open("hidden_rankings_hidden1.pkl", "rb"))
results_data = np.zeros((len(test_predictions),2))
print(len(test_predictions))
for i in range(len(test_predictions)):
    current_pred = test_predictions[i]
    results_data[i, 0] = current_pred.est
    results_data[i, 1] = current_pred.r_ui
test_rmse = mean_squared_error(results_data[:, 1], results_data[:, 0], squared = False)
print(test_rmse)

4236
0.8092063659409897


## RMSE on all the test ingredients (with 1 hidden ingredient)

In [5]:
test_predictions = pickle.load(open("rec_predictions_hidden1.pkl", "rb"))
hidden_rankings = pickle.load(open("rec_rankings_hidden1.pkl", "rb"))
print(len(test_predictions))

148199


In [6]:
results_data = np.zeros((len(test_predictions),2))
print(len(test_predictions))
for i in range(len(test_predictions)):
    current_pred = test_predictions[i]
    results_data[i, 0] = current_pred.est
    results_data[i, 1] = current_pred.r_ui
test_rmse = mean_squared_error(results_data[:, 1], results_data[:, 0], squared = False)
print(test_rmse)

148199
0.06659291134285636


## Recommendations

In [7]:
recommendations = {}
for pred in test_predictions:
    current_rec = recommendations.get(pred.uid, ())
    if len(current_rec) > 0:
        max_rating = current_rec[1]
        if pred.est > max_rating:
            recommendations[pred.uid] = (pred.iid, pred.est)
    else:
        recommendations[pred.uid] = (pred.iid, pred.est)

In [10]:
accuracy = 0
correct_examples = []
for recipe in recommendations:
    true_ingredient = hidden_ingredients.loc[hidden_ingredients['recipe_id'] == recipe]['ingredient']
    predicted_ingredient = recommendations[recipe][0]
    if true_ingredient.item() == predicted_ingredient:
        correct_examples.append(recipe)
        accuracy += 1
print("Correct ingredients: ")
print(accuracy)
print("Test recipes: ")
print(len(recommendations))
print("Accuracy: ")
print(accuracy / len(recommendations))

Correct ingredients: 
12
Test recipes: 
212
Accuracy: 
0.05660377358490566


## Correct Examples

In [11]:
full_testset = pickle.load(open("full_test_hidden1.pkl", "rb"))
full_testset.head()

,recipe_id,ingredient,rating
0,0,'acorn squash',0
1,1,'acorn squash',0
2,2,'acorn squash',0
3,3,'acorn squash',0
4,4,'acorn squash',0


In [12]:
active_ingredients = full_testset.loc[full_testset['rating'] == 1]
for recipe in correct_examples:
    print("Recipe: ")
    print(recipe)
    ingredients = active_ingredients.loc[active_ingredients['recipe_id'] == recipe]
    print("Ingredients: ")
    print(ingredients['ingredient'].to_string(index = False))
    predicted_ingredient = recommendations[recipe]
    print("Recommended ingredient: ")
    print(predicted_ingredient[0])

Recipe: 
4
Ingredients: 
 'amchoor powder'
   'chile pepper'
   'chili powder'
        coriander
            cumin
        fenugreek
           garlic
           tomato
         turmeric
Recommended ingredient: 
'vegetable oil'
Recipe: 
6
Ingredients: 
       'balsamic vinegar'
                    basil
            'brown sugar'
 'Italian salad dressing'
              'olive oil'
                    onion
                  parsley
                   pepper
                     salt
                  sausage
                   tomato
           'tomato paste'
                    water
Recommended ingredient: 
garlic
Recipe: 
42
Ingredients: 
         broth
 'brown sugar'
 'chili paste'
    cornstarch
           egg
       ketchup
   'olive oil'
          salt
   'soy sauce'
         sugar
       vinegar
Recommended ingredient: 
chicken
Recipe: 
63
Ingredients: 
    broth
   butter
  chicken
    flour
 mushroom
    olive
    onion
     wine
Recommended ingredient: 
'vegetable oil'
Recipe

### Incorrect examples

In [15]:
counter = 0
for recipe in recommendations:
    if counter > 25:
        break
    print("Recipe: ")
    print(recipe)
    ingredients = active_ingredients.loc[active_ingredients['recipe_id'] == recipe]
    print("Ingredients: ")
    print(ingredients['ingredient'].to_string(index = False))
    predicted_ingredient = recommendations[recipe]
    print("Recommended ingredient: ")
    print(predicted_ingredient[0])
    print("True ingredient: ")
    true_ingredient = hidden_ingredients.loc[hidden_ingredients['recipe_id'] == recipe]['ingredient']
    print(true_ingredient.item())
    counter += 1

Recipe: 
0
Ingredients: 
      cilantro
           egg
        garlic
        ginger
 'green onion'
 'lemon grass'
          lime
        turkey
         water
Recommended ingredient: 
'soy sauce'
True ingredient: 
'chile pepper'
Recipe: 
1
Ingredients: 
          'chile pepper'
                cilantro
 'coconut milk or cream'
               coriander
                   cumin
                  garlic
                  ginger
                    lime
                   onion
                turmeric
            'white fish'
Recommended ingredient: 
'soy sauce'
True ingredient: 
masala
Recipe: 
2
Ingredients: 
     'cider vinegar'
          cornstarch
                 egg
       'green onion'
             ketchup
               onion
           pineapple
                pork
 'red pepper flakes'
       'sesame seed'
         'soy sauce'
               sugar
      'sweet pepper'
     'vegetable oil'
               water
Recommended ingredient: 
garlic
True ingredient: 
salt
Recipe: 
3
In